In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
from multiprocess import Pool
import numpy as np

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from conversion import images_to_volume

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt


In [2]:
classifier_id = 30

In [3]:
downscale = 32
voxel_z_size = SECTION_THICKNESS/(XY_PIXEL_DISTANCE_LOSSLESS * downscale)

In [4]:
def load_downscaled_scoremaps_multiple_sections_sequential(sections, stack, structure, downscale, classifier_id):
    scoremaps = {}
    for sec in sections:
        try:
            actual_setting = resolve_actual_setting(setting=classifier_id, stack=stack, sec=sec)
            sm = DataManager.load_downscaled_scoremap(stack=stack, section=sec, structure=structure, downscale=downscale, setting=actual_setting)
            if sm is not None:
                scoremaps[sec] = sm
        except:
            pass
    return scoremaps

# Alternating

In [5]:
for stack in ['MD657']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]

    nissl_sections = []
    for sec in range(first_sec, last_sec):
        fn = metadata_cache['sections_to_filenames'][stack][sec]
        if not is_invalid(fn) and fn.split('-')[1][0] == 'N':
            nissl_sections.append(sec)
    
    for structure in all_known_structures:
#     for structure in ['7N']:
        
        print structure
    
        t = time.time()
        scoremaps = load_downscaled_scoremaps_multiple_sections_sequential(stack=stack, sections=nissl_sections, 
                                                              structure=structure, downscale=downscale, 
                                                              classifier_id=classifier_id)
        print len(scoremaps), 'scoremaps loaded.'
        if len(scoremaps) < 2:
            sys.stderr.write('Number of valid scoremaps for %s is less than 2.\n' % structure)
            continue
        
        sys.stderr.write('Load scoremaps: %.2f seconds\n' % (time.time() - t)) # 10-40s (down=32, 12 processes)
        
        t = time.time()
        score_volume, score_volume_bbox = images_to_volume(images=scoremaps, voxel_size=(1, 1, voxel_z_size), 
                                                           first_sec=np.min(nissl_sections)-1, 
                                                           last_sec=np.max(nissl_sections)-1)
        sys.stderr.write('Create score volume: %.2f seconds\n' % (time.time() - t)) # 6s (sequential)
        
        score_volume_filepath = DataManager.get_score_volume_filepath(stack=stack, downscale=downscale, 
                                                                      structure=structure, 
                                                                      classifier_setting=classifier_id)
        create_parent_dir_if_not_exists(score_volume_filepath)
        bp.pack_ndarray_file(score_volume.astype(np.float16), score_volume_filepath)

        score_volume_bbox_filepath = DataManager.get_score_volume_bbox_filepath(stack=stack, downscale=downscale, structure=structure,
                                                                               classifier_setting=classifier_id)
        np.savetxt(score_volume_bbox_filepath, np.array(score_volume_bbox)[None], fmt='%d')
        
        upload_from_ec2_to_s3(score_volume_filepath)
        upload_from_ec2_to_s3(score_volume_bbox_filepath)
        
        del score_volume, scoremaps

5N
rm -rf /shared/CSHL_scoremaps_down32/MD657/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32_5N_denseScoreMap_setting_30.bp && mkdir -p /shared/CSHL_scoremaps_down32/MD657/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32
aws s3 cp s3://mousebrainatlas-data/CSHL_scoremaps_down32/MD657/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32_5N_denseScoreMap_setting_30.bp /shared/CSHL_scoremaps_down32/MD657/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.0

Child returned 0
Child returned 1
0.35 seconds.
Child returned 0


rm -rf /shared/CSHL_scoremaps_down32/MD657/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32_5N_denseScoreMap_setting_30.bp && mkdir -p /shared/CSHL_scoremaps_down32/MD657/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32
aws s3 cp s3://mousebrainatlas-data/CSHL_scoremaps_down32/MD657/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32_5N_denseScoreMap_setting_30.bp /shared/CSHL_scoremaps_down32/MD657/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.2

Child returned 1
0.35 seconds.
Load scoremaps: 0.87 seconds


127 scoremaps loaded.


...

Create score volume: 2.28 seconds


aws s3 cp /shared/CSHL_volumes/MD657/MD657_down32_scoreVolume_clf_30/score_volumes/MD657_down32_scoreVolume_clf_30_IC.bp s3://mousebrainatlas-data/CSHL_volumes/MD657/MD657_down32_scoreVolume_clf_30/score_volumes/MD657_down32_scoreVolume_clf_30_IC.bp


Child returned 0
1.46 seconds.


aws s3 cp /shared/CSHL_volumes/MD657/MD657_down32_scoreVolume_clf_30/score_volumes/MD657_down32_scoreVolume_clf_30_IC_bbox.txt s3://mousebrainatlas-data/CSHL_volumes/MD657/MD657_down32_scoreVolume_clf_30/score_volumes/MD657_down32_scoreVolume_clf_30_IC_bbox.txt


Child returned 0
0.36 seconds.


# Regular

In [24]:
for stack in ['MD591']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    
    for structure in all_known_structures:
#     for structure in ['7N']:
        print structure
    
        t = time.time()
#         scoremaps = load_scoremaps_multiple_sections_parallel(stack=stack, sections=range(first_sec, last_sec+1), 
#                                                         structure=structure, downscale=downscale, classifier_setting=classifier_id)
        scoremaps = load_downscaled_scoremaps_multiple_sections_sequential(stack=stack, sections=range(first_sec, last_sec+1), 
                                                structure=structure, downscale=downscale, classifier_setting=classifier_id)
        
        if len(scoremaps) < 2:
            sys.stderr.write('Number of valid scoremaps for %s is less than 2.\n' % structure)
            continue
        
        sys.stderr.write('Load scoremaps: %.2f seconds\n' % (time.time() - t)) # 10-40s (down=32, 12 processes)
        
        t = time.time()
        score_volume, score_volume_bbox = images_to_volume(images=scoremaps, voxel_size=(1, 1, voxel_z_size), 
                                                           first_sec=first_sec-1, last_sec=last_sec-1)
        sys.stderr.write('Create score volume: %.2f seconds\n' % (time.time() - t)) # 2s
        
#         t = time.time()

        score_volume_filepath = DataManager.get_score_volume_filepath(stack=stack, downscale=downscale, structure=structure, classifier_setting=classifier_id)
        create_parent_dir_if_not_exists(score_volume_filepath)
        bp.pack_ndarray_file(score_volume.astype(np.float16), score_volume_filepath)

        score_volume_bbox_filepath = DataManager.get_score_volume_bbox_filepath(stack=stack, downscale=downscale, structure=structure,
                                                                               classifier_setting=classifier_id)
        np.savetxt(score_volume_bbox_filepath, np.array(score_volume_bbox)[None], fmt='%d')
        
        upload_from_ec2_to_s3(score_volume_filepath)
        upload_from_ec2_to_s3(score_volume_bbox_filepath)
        
        del score_volume, scoremaps
        
#         sys.stderr.write('Save score volume: %.2f seconds\n' % (time.time() - t)) # 1s (down=32)

5N


Load scoremaps: 1.04 seconds
Create score volume: 1.50 seconds


aws s3 cp /shared/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volumes/MD591_down32_scoreVolume_clf_30_5N.bp s3://mousebrainatlas-data/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volumes/MD591_down32_scoreVolume_clf_30_5N.bp


Child returned 0
1.11 seconds.


aws s3 cp /shared/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volumes/MD591_down32_scoreVolume_clf_30_5N_bbox.txt s3://mousebrainatlas-data/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volumes/MD591_down32_scoreVolume_clf_30_5N_bbox.txt


...

Load scoremaps: 1.04 seconds
Create score volume: 2.04 seconds


aws s3 cp /shared/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volumes/MD591_down32_scoreVolume_clf_30_IC.bp s3://mousebrainatlas-data/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volumes/MD591_down32_scoreVolume_clf_30_IC.bp


Child returned 0
1.03 seconds.


aws s3 cp /shared/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volumes/MD591_down32_scoreVolume_clf_30_IC_bbox.txt s3://mousebrainatlas-data/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volumes/MD591_down32_scoreVolume_clf_30_IC_bbox.txt


Child returned 0
0.38 seconds.


# Read

In [25]:
for structure in all_known_structures:
    print structure
    try:
        score_volume = DataManager.load_score_volume(stack='MD642', structure=structure, downscale=32, 
                                                 classifier_setting=classifier_setting)
        score_volume_cropped = score_volume[:, :600, :]

        score_volume_filepath = DataManager.get_score_volume_filepath(stack=stack, downscale=downscale, 
                                                                  structure=structure, 
                                                                  classifier_setting=classifier_setting)
        create_if_not_exists(os.path.dirname(score_volume_filepath))
        bp.pack_ndarray_file(score_volume_cropped.astype(np.float16), score_volume_filepath)
    except:
        pass

5N
6N
7N
7n
Amb
LC
LRt
Pn
Tz
VLL
RMC
SNC
SNR
3N
4N
Sp5I
Sp5O
Sp5C
PBG
10N
VCA
VCP
DC
AP
12N
RtTg
sp5
outerContour
SC


File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD642/MD642_down32_scoreVolume_clf_2/score_volumes/MD642_down32_scoreVolume_clf_2_sp5.bp
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD642/MD642_down32_scoreVolume_clf_2/score_volumes/MD642_down32_scoreVolume_clf_2_outerContour.bp


IC


In [ ]:
display_volume_sections(score_volume_cropped, cmap=plt.cm.hot, every=10)